# Notebook Description

Although oddly similar to the Finance Trading Strategies file, this notebook optimizes one of the strategies in that file. This notebook optimizes the 3 SMA Trading Strategy utilizing a different company from the portfolio found in the Finance Trading Strategies file but over the same data range. This notebook finds the optimial combination of the shortest, medium, and largest simple moving averages to produce the greatest return amount. As of now, the company utilized to backtest the strategy is Apple (AAPL).

Data Source: Microsoft Excel

Date Range: January 1, 2017 to July 26, 2023

# Instructions For Use

 Optimization Code at the bottom of the notebook!
 
 - Step 1: Run the Imports, SMA, Results, and Load
 - Step 2: Now view the "Looking at Whole History"
 - Step 3: Adjust the range of x, y, and z values as you see fit
 - Step 4: Run the cell and wait as there are 2 nested for loops, it will take a while for the computation
 - Step 5: Your optimized X, Y, Z values will be presented for the given backtested company (Apple as of now).

# Imports

In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# SMA

In [2]:
def SMA(df, x, y, z):
    # Create the SMA values for the intervals of x days, y days, and z days
    df["small SMA"] = df["Close"].rolling(window=x).mean()
    df["int SMA"] = df["Close"].rolling(window=y).mean()
    df["large SMA"] = df["Close"].rolling(window=z).mean()

    # Trading Strategy CODE
    # 3 MA strategy (big, intermediate, and small)
    df["Signal"] = np.where((df["small SMA"] > df["int SMA"]) & (df["int SMA"] > df["large SMA"]), "Buy",
                            np.where((df["large SMA"] > df["int SMA"]) & (df["int SMA"] > df["small SMA"]), "Sell", "Hold"))

    # 2 MA strategy (big and small)
    # df["Signal"] = np.where((df["small SMA"] > df["large SMA"]), "Buy",
    #                         np.where((df["large SMA"] > df["small SMA"]), "Sell", "Hold"))

    # BUY/SELL SIGNAL CODE 
    # DISCLAIMER NOTE: This specific buy/sell signal code in this cell is not my own work. This code was generated through outside resources (ChatGPT) and it is not my work. The rest of the code in the Notebook and Cell IS indeed my own work.
    buy_date = None
    buy_price = None
    sell_date = None
    sell_price = None
    ignore_signals = False

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if ignore_signals:
            if row["Signal"] == "Buy":
                ignore_signals = False
        else:
            if row["Signal"] == "Buy" and buy_date is None:
                buy_date = index
                buy_price = row["Open"]
                ignore_signals = True
            elif row["Signal"] == "Sell" and buy_date is not None and sell_date is None:
                sell_date = df.index[df.index.get_loc(index) + 1]
                sell_price = df.loc[sell_date, "Open"]
                ignore_signals = True
                # Calculate the +/- amount and update the '+/- Amount' column
                plus_minus_amount = sell_price - buy_price
                df.loc[sell_date, "+/- Amount"] = plus_minus_amount
                # Reset the variables for the next pair of signals
                buy_date = None
                buy_price = None
                sell_date = None
                sell_price = None
    return df


# Results

In [3]:
def results(df):
    # Breakdown of All trades
    results_df = df.dropna().copy()
    results_df.loc[:, "+/- Amount"] = results_df.loc[:, "+/- Amount"].round(2)
    
    # Breakdown of Gains
    gain_df = results_df[results_df["+/- Amount"] > 0]
    
    # Breakdown of Losses
    loss_df = results_df[results_df["+/- Amount"] < 0]

    # Analysis and Ratios of Tables
    ratio = {
        "Final Amount": results_df["+/- Amount"].sum().round(2),
        "Number of Trades": len(results_df),
        "Win Ratio": [round(len(gain_df)/len(results_df), 2) * 100],
        "Loss Ratio": [round(len(loss_df)/len(results_df), 2) * 100],
        "Average Gain per Trade": [round(gain_df["+/- Amount"].mean(), 2)],
        "Average Loss per Trade": [round(loss_df["+/- Amount"].mean(), 2)],
        "Final Ratio": [abs(round((gain_df["+/- Amount"].sum() / len(gain_df)) / (loss_df["+/- Amount"].sum() / len(loss_df)), 2))]
    }
    ratio_df = pd.DataFrame(ratio)

    return results_df, gain_df, loss_df, ratio_df

# Load

In [4]:
df = pd.read_excel("Apple Price.xlsx")

# Drop 0th row, make 1st row headers, and values in 2nd row
df = pd.DataFrame(df.values[2:], columns=df.iloc[1])

#Make All Columns Numeric
df["Open"] = pd.to_numeric(df["High"], errors="coerce")
df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
df["Low"] = pd.to_numeric(df["Low"], errors="coerce")
df["High"] = pd.to_numeric(df["High"], errors="coerce")


# Change Date Column from imported to proper format and set as index
df["Date"] = pd.to_numeric(df["Date"], errors="coerce")
df["Date"] = pd.to_datetime(df["Date"], origin="1899-12-30", unit="D").dt.strftime("%m/%d/%y")
df = df.set_index("Date")

# Reorder columns
desired = ["Open", "Close", "Low", "High"]
df = df.reindex(columns=desired)


# Calculate SMA and perform trading strategy
df = SMA(df, 5, 10, 20)

 # Calculate results for each company
results_df, gain_df, loss_df, ratio_df = results(df)

print("The total account would be up: $", df["+/- Amount"].sum().round(2), "for 1 share using this trading strategy")
print("")
print("Analysis of Trading Strategy:")
display(ratio_df)
print("Breakdown of All Trades:")
display(results_df)
print("Breakdown of All Gains:")
display(gain_df)
print("Breakdown of All Losses:")
display(loss_df)

# Append results to the outcome dictionary
outcome_dict = {
    "Final Amount": results_df["+/- Amount"].sum().round(2),
    "Win Ratio": ratio_df.iloc[0]["Win Ratio"],
    "Loss Ratio": ratio_df.iloc[0]["Loss Ratio"],
    "Average Gain per Trade": ratio_df.iloc[0]["Average Gain per Trade"],
    "Average Loss per Trade": ratio_df.iloc[0]["Average Loss per Trade"],
    "Final Ratio": ratio_df.iloc[0]["Final Ratio"]
}

# Convert the outcome dictionary to a DataFrame
outcome_df = pd.DataFrame.from_dict(outcome_dict, orient="index")

# Display the outcome DataFrame for all companies
print("Outcome DataFrame for All Companies:")
display(outcome_df)


The total account would be up: $ 116.06 for 1 share using this trading strategy

Analysis of Trading Strategy:


,Final Amount,Number of Trades,Win Ratio,Loss Ratio,Average Gain per Trade,Average Loss per Trade,Final Ratio
0,116.09,28,54.0,46.0,12.33,-5.29,2.33


Breakdown of All Trades:


1,Open,Close,Low,High,small SMA,int SMA,large SMA,Signal,+/- Amount
Date,,,,,,,,,
04/20/17,35.7300,35.6100,35.2900,35.7300,35.3600,35.52000,35.640500,Sell,5.38
06/12/17,36.5225,36.3550,35.6275,36.5225,37.9605,38.20500,38.297875,Sell,-0.50
09/19/17,39.9425,39.6825,39.6100,39.9425,39.7605,39.98425,40.201500,Sell,2.22
12/04/17,43.1550,42.4500,42.4075,43.1550,42.7625,43.05975,43.173500,Sell,3.84
01/08/18,43.9025,43.5875,43.4825,43.9025,43.3435,43.08400,43.198500,Hold,-0.10
01/31/18,42.1104,41.8575,41.6250,42.1104,42.2490,43.27400,43.523125,Sell,-2.74
03/23/18,42.4800,41.2350,41.2350,42.4800,42.7800,43.81000,44.056625,Sell,-1.43
04/27/18,41.0825,40.5800,40.1575,41.0825,40.9185,42.21975,42.515750,Sell,-2.96
06/22/18,46.5375,46.2300,46.1750,46.5375,46.5655,47.12900,47.381625,Sell,-0.31


Breakdown of All Gains:


1,Open,Close,Low,High,small SMA,int SMA,large SMA,Signal,+/- Amount
Date,,,,,,,,,
04/20/17,35.7300,35.6100,35.2900,35.7300,35.3600,35.52000,35.640500,Sell,5.38
09/19/17,39.9425,39.6825,39.6100,39.9425,39.7605,39.98425,40.201500,Sell,2.22
12/04/17,43.1550,42.4500,42.4075,43.1550,42.7625,43.05975,43.173500,Sell,3.84
09/20/18,55.5700,55.0075,54.7875,55.5700,54.9180,55.23300,55.407250,Sell,7.61
05/14/19,47.4250,47.1650,46.3525,47.4250,48.7590,50.44850,50.727625,Sell,8.64
08/08/19,50.8825,50.8575,49.8475,50.8825,49.8415,51.11250,51.260750,Sell,1.69
02/26/20,74.4700,73.1625,71.6250,74.4700,75.6130,78.29750,79.062250,Sell,20.86
09/15/20,118.8290,115.5400,113.6100,118.8290,114.7410,119.39450,121.196125,Sell,51.31
10/29/20,116.9300,115.3200,112.2000,116.9300,114.6420,115.83400,116.772000,Sell,0.81


Breakdown of All Losses:


1,Open,Close,Low,High,small SMA,int SMA,large SMA,Signal,+/- Amount
Date,,,,,,,,,
06/12/17,36.5225,36.3550,35.6275,36.5225,37.9605,38.20500,38.297875,Sell,-0.50
01/08/18,43.9025,43.5875,43.4825,43.9025,43.3435,43.08400,43.198500,Hold,-0.10
01/31/18,42.1104,41.8575,41.6250,42.1104,42.2490,43.27400,43.523125,Sell,-2.74
03/23/18,42.4800,41.2350,41.2350,42.4800,42.7800,43.81000,44.056625,Sell,-1.43
04/27/18,41.0825,40.5800,40.1575,41.0825,40.9185,42.21975,42.515750,Sell,-2.96
06/22/18,46.5375,46.2300,46.1750,46.5375,46.5655,47.12900,47.381625,Sell,-0.31
10/18/18,54.9350,54.0050,53.2500,54.9350,54.9415,55.11425,55.661500,Sell,-3.43
02/17/21,132.2200,130.8400,129.4700,132.2200,133.9840,135.09300,136.261000,Sell,-12.08
09/21/21,144.6000,143.4300,142.7800,144.6000,146.0500,148.60700,150.114500,Sell,-6.59


Outcome DataFrame for All Companies:


,0
Final Amount,116.09
Win Ratio,54.00
Loss Ratio,46.00
Average Gain per Trade,12.33
Average Loss per Trade,-5.29
Final Ratio,2.33


# Looking at Whole History

In [5]:
def SMA(df, x, y, z):
    # Create the SMA values for the intervals of x days, y days, and z days
    df["small SMA"] = df["Close"].rolling(window=x).mean()
    df["int SMA"] = df["Close"].rolling(window=y).mean()
    df["large SMA"] = df["Close"].rolling(window=z).mean()

    # Trading Strategy CODE
    # 3 MA strategy (big, intermediate, and small)
    df["Signal"] = np.where((df["small SMA"] > df["int SMA"]) & (df["int SMA"] > df["large SMA"]), "Buy",
                            np.where((df["large SMA"] > df["int SMA"]) & (df["int SMA"] > df["small SMA"]), "Sell", "Hold"))

    # BUY/SELL SIGNAL CODE 
    # DISCLAIMER NOTE: This specific buy/sell signal code in this cell is not my own work. This code was generated through outside resources (ChatGPT) and it is not my work. The rest of the code in the Notebook and Cell IS indeed my own work.
    buy_date = None
    buy_price = None
    sell_date = None
    sell_price = None
    ignore_signals = False

    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if ignore_signals:
            if row["Signal"] == "Buy":
                ignore_signals = False
        else:
            if row["Signal"] == "Buy" and buy_date is None:
                buy_date = index
                buy_price = row["Open"]
                ignore_signals = True
            elif row["Signal"] == "Sell" and buy_date is not None and sell_date is None:
                sell_date = df.index[df.index.get_loc(index) + 1]
                sell_price = df.loc[sell_date, "Open"]
                ignore_signals = True
                # Calculate the +/- amount and update the '+/- Amount' column
                plus_minus_amount = sell_price - buy_price
                df.loc[sell_date, "+/- Amount"] = plus_minus_amount
                # Reset the variables for the next pair of signals
                buy_date = None
                buy_price = None
                sell_date = None
                sell_price = None
    return df


#Define the range of values to test for x, y, and z
x_values = range(0, 6)  # Example range: 10 to 20
y_values = range(6, 16)  # Example range: 20 to 40
z_values = range(16, 30)  # Example range: 40 to 70

best_performance = float('-inf')
best_combination = None

#Iterate through all combinations of x, y, and z values
for x in x_values:
    for y in y_values:
        for z in z_values:
            df_copy = df.copy() 
            #Apply the SMA function with the current combination of x, y, and z values
            df_copy = SMA(df_copy, x, y, z)
            performance = df_copy["+/- Amount"].sum()
            #Check if the current combination yields the best performance so far
            if performance > best_performance:
                best_performance = performance
                best_combination = (x, y, z)

print("Best Combination (x, y, z):", best_combination)
print("Best Performance:", best_performance)

Best Combination (x, y, z): (3, 15, 18)
Best Performance: 270.9106


# Looking at just 2023

In [6]:
# Convert the index to a datetime type if it is not already in that format
df.index = pd.to_datetime(df.index)
# Filter the DataFrame based on the index values
df_filtered = df[df.index >= pd.to_datetime("2023-01-01")]

def SMA(df, x, y, z):
    # Create the SMA values for the intervals of x days, y days, and z days
    df["small SMA"] = df["Close"].rolling(window=x).mean()
    df["int SMA"] = df["Close"].rolling(window=y).mean()
    df["large SMA"] = df["Close"].rolling(window=z).mean()

    #Trading Strategy CODE
    #3 MA strategy (big, intermediate, and small)
    df["Signal"] = np.where((df["small SMA"] > df["int SMA"]) & (df["int SMA"] > df["large SMA"]), "Buy",
                            np.where((df["large SMA"] > df["int SMA"]) & (df["int SMA"] > df["small SMA"]), "Sell", "Hold"))

    # BUY/SELL SIGNAL CODE 
    # DISCLAIMER NOTE: This specific buy/sell signal code in this cell is not my own work. This code was generated through outside resources (ChatGPT) and it is not my work. The rest of the code in the Notebook and Cell IS indeed my own work.
    buy_date = None
    buy_price = None
    sell_date = None
    sell_price = None
    ignore_signals = False

    #Iterate over the DataFrame rows
    for index, row in df.iterrows():
        if ignore_signals:
            if row["Signal"] == "Buy":
                ignore_signals = False
        else:
            if row["Signal"] == "Buy" and buy_date is None:
                buy_date = index
                buy_price = row["Open"]
                ignore_signals = True
            elif row["Signal"] == "Sell" and buy_date is not None and sell_date is None:
                sell_date = df.index[df.index.get_loc(index) + 1]
                sell_price = df.loc[sell_date, "Open"]
                ignore_signals = True
                #Calculate the +/- amount and update the '+/- Amount' column
                plus_minus_amount = sell_price - buy_price
                df.loc[sell_date, "+/- Amount"] = plus_minus_amount
                #Reset the variables for the next pair of signals
                buy_date = None
                buy_price = None
                sell_date = None
                sell_price = None
    return df

#Define the range of values to test for x, y, and z
x_values = range(0, 6)  # Example range: 10 to 20
y_values = range(6, 16)  # Example range: 20 to 40
z_values = range(16, 30)  # Example range: 40 to 70

best_performance = float('-inf')
best_combination = None

#Iterate through all combinations of x, y, and z values
for x in x_values:
    for y in y_values:
        for z in z_values:
            #Use the filtered DF
            df_copy = df_filtered.copy() 
            df_copy = SMA(df_copy, x, y, z)
            performance = df_copy["+/- Amount"].sum()  
            #Check if the current combination yields the best performance so far
            if performance > best_performance:
                best_performance = performance
                best_combination = (x, y, z)

print("Best Combination (x, y, z):", best_combination)
print("Best Performance:", best_performance)

Best Combination (x, y, z): (4, 14, 16)
Best Performance: 43.46349999999998
